In [11]:
%matplotlib inline 
import sys
from collections import OrderedDict
import numpy as np
import pandas as pd

sys.path.append("../")
from utils import FullIndex, ms_table


#t = 'ceededdbe'
#s = 'cbeeeddba'
s = 'bcdbaecdcd'
t = 'dcdceabdcbdcdceabdcb'

s_index = FullIndex(s)

for i in range(len(s)):
    for j in range(i, len(s)):
        substr = s[i:j]
        a, b = s_index.sa_interval(substr, FullIndex.FWD)
        print (a, b, s[i:j], 
               s_index.is_left_maximal(a, b), s_index.is_right_maximal(a, b), 
               ' --> ', s_index.is_maximal(a, b))


(0, 11, '', True, True, ' --> ', True)
(2, 4, 'b', True, True, ' --> ', True)
(3, 4, 'bc', False, False, ' --> ', False)
(3, 4, 'bcd', False, False, ' --> ', False)
(3, 4, 'bcdb', False, False, ' --> ', False)
(3, 4, 'bcdba', False, False, ' --> ', False)
(3, 4, 'bcdbae', False, False, ' --> ', False)
(3, 4, 'bcdbaec', False, False, ' --> ', False)
(3, 4, 'bcdbaecd', False, False, ' --> ', False)
(3, 4, 'bcdbaecdc', False, False, ' --> ', False)
(0, 11, '', True, True, ' --> ', True)
(4, 7, 'c', True, False, ' --> ', False)
(4, 7, 'cd', True, False, ' --> ', False)
(5, 6, 'cdb', False, False, ' --> ', False)
(5, 6, 'cdba', False, False, ' --> ', False)
(5, 6, 'cdbae', False, False, ' --> ', False)
(5, 6, 'cdbaec', False, False, ' --> ', False)
(5, 6, 'cdbaecd', False, False, ' --> ', False)
(5, 6, 'cdbaecdc', False, False, ' --> ', False)
(0, 11, '', True, True, ' --> ', True)
(7, 10, 'd', False, True, ' --> ', False)
(8, 9, 'db', False, False, ' --> ', False)
(8, 9, 'dba', False, Fals

In [3]:
#tt = ms_table(t, s)
#tt['pre_merge_RUNS'] = [-1] + [0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,1]
#tt['error'] = map(int, tt.pre_merge_RUNS != tt.runs)
#tt#[tt.error == 1]

In [4]:
#table2(s)#[50:80]
#index_table(s)
s_index.tabs[FullIndex.FWD]

,s_i,BWT,SA,suff_SA
i,,,,
0,b,d,10,#
1,c,b,4,aecdcd#
2,d,d,3,baecdcd#
3,b,#,0,bcdbaecdcd#
4,a,d,8,cd#
5,e,b,1,cdbaecdcd#
6,c,e,6,cdcd#
7,d,c,9,d#
8,c,c,2,dbaecdcd#


In [8]:
s_index.tabs[FullIndex.REV]

,s_i,BWT,SA,suff_SA
i,,,,
0,b,d,10,#
1,c,b,4,aecdcd#
2,d,d,3,baecdcd#
3,b,#,0,bcdbaecdcd#
4,a,d,8,cd#
5,e,b,1,cdbaecdcd#
6,c,e,6,cdcd#
7,d,c,9,d#
8,c,c,2,dbaecdcd#


In [10]:
t_fwd = table2(s)
t_rev = table2(s[::-1])

def sa_interval(tab, query):
    pref_bool_idx = tab.suff_SA.apply(lambda suff: suff.startswith(query))
    idx = tab.loc[pref_bool_idx].index.tolist()
    return min(idx), max(idx) + 1

def is_maximal(tab, query):
    ij = sa_interval(tab, query)
    return len(set(tab[ij[0]:ij[1]].BWT)) > 1


NameError: name 'table2' is not defined